# Random Forest - Poisonous Mushroom Classifier

Have you ever picked up wild mushroom and wondered whether it was edible or not? We are given a dataset of purely categorical features and want to create a model to classify whether a mushroom with specific traits is edible or poisonous. 
The data guide states that there is no simple rule to determine whether a mushroom is poisonous or not. This is why an ML model would be beneficial.

# 0. Import Libraries and Dataset

In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import chi2_contingency

from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import os

data_path = '../input/mushroom-classification/mushrooms.csv'
mushroom_data = pd.read_csv(data_path)

# 1. Exploratory Data Analysis

We can start by taking a peak at the dataset and how it is formatted.

In [ ]:
mushroom_data.info()

In [ ]:
mushroom_data.head()

In [ ]:
# Shape of the dataset
print("We have", mushroom_data.shape[1], "features with", mushroom_data.shape[0], "observations")

mushroom_data.describe().T

In [ ]:
# Number of missing data values
print("The dataset has", mushroom_data.isnull().sum().sum(), "missing points.")

mushroom_data.isnull().sum()

In [ ]:
# object_columns = [col for col in mushroom_data.columns if mushroom_data[col].dtype==object]
mushroom_data.dtypes

In [ ]:
class_data = mushroom_data['class'].value_counts()
pie, ax = plt.subplots(figsize=[10,6])
labels = class_data.keys()
plt.pie(x=class_data, autopct="%.1f%%",explode=[0,0.05], labels=labels, pctdistance=0.5)
plt.title("Edible vs Poisonous Mushrooms in Dataset", fontsize=14)

### Summary

We have 8124 observations with 23 features. One of the features will be used for our target variable (class). All our features are object types and are not missing any data points.

### Initial Thoughts

Since we have a relatively small dataset here, I am thinking of one-hot encoding all the features. If we did have missing values, I would experiment with removing rows or imputing and how that would affect the model.
I also noticed that there a feature that has one value for all 8124 observations (veil-type). I will probably end up dropping that column from the dataset.

# 2. Feature Engineering and Selection

Because all of our features are categorical along with our target variable, we have two main tests for feature importance

1. Chi-squared test
2. Mutual Information

### One-Hot Encoding (getting data ready for analysis)

In [ ]:
# Copy the original data
X = mushroom_data.copy()
y = X.pop('class')

# Remove useless feature
X = X.drop('veil-type', axis=1)

X = pd.get_dummies(X)
y = pd.factorize(y)

X.head()

### What is the Chi-squared test for feature selection?

The Chi-squared test for independence is for testing whether two variables are independent from each other. When working with categorical data, the chi-square test is often used to minimize the noise in the input when building the model. 

\begin{gather*}
H_0 = \text{The relationship is independent}\\
H_1 = \text{The relationship is not independent}
\end{gather*}

The p-value we will use as our threshold is 0.05. That means we are looking at a 95% confidence interval. However one thing to note is that we are one-hot encoding the features before running the chi-squared tests. Since we are doing it this way, 
the p-value that we are going to use to reject the null hypothesis is going to be different depending on the feature's number of unique instances. The p-value we will actually use will be determined by the following formula:

\begin{gather*}
\text{p-value} = \frac{0.05}{\text{# of unique indexes}}
\end{gather*}

In [ ]:
X_temp = mushroom_data.copy().drop('class', axis=1).drop('veil-type', axis=1)
# Find the p values from the chi2 test for each column
chi2_scores = []
chi2_pvalues = []
features_to_drop = []
for i in X_temp.columns:
    dummies = pd.get_dummies(X_temp[i])
    for j in dummies:
        chi_score = chi2_contingency(pd.crosstab(y[0],dummies[j]))[0]
        p_value = chi2_contingency(pd.crosstab(y[0],dummies[j]))[1]*X_temp[i].nunique()
        chi2_scores.append(chi_score)
        chi2_pvalues.append(p_value)
        if p_value > 0.05:
            features_to_drop.append(i+'_'+j)
chi2_scores = pd.Series(chi2_scores, index=X.columns).sort_values(ascending=False)
chi2_scores

In [ ]:
# Plot the Chi2 scores
fig, ax = plt.subplots(figsize=(15,25))

sns.barplot(x = chi2_scores, y=chi2_scores.index)

ax.set_xlabel('Chi-Squared Scores')
ax.set_ylabel('Features')
ax.set_title('Chi-Squared Scores of Features')

In [ ]:
# These are the features that failed to reject the null hypothesis (p-value>0.05)
features_to_drop

In [ ]:
# Dropping failed features
X_chi2 = X.drop(features_to_drop, axis=1)
X_chi2.head()

### Mutual Information method of Feature Selection

Mutual Information measures the non-linear relationship between two variables. We will be calculating a score for mutual information and build a model with the selected features.

In [ ]:
# Mutual Information
mi_scores = mutual_info_classif(X, y[0])
mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

In [ ]:
# Plot the Chi2 scores
fig, ax = plt.subplots(figsize=(15,25))

sns.barplot(x = mi_scores, y=mi_scores.index)

ax.set_xlabel('Mutual Information Scores')
ax.set_ylabel('Features')
ax.set_title('MI Score')

In [ ]:
# Lets find redundant and non-relevant features
duplicates = mi_scores.duplicated()

bad_features = [x for x in duplicates.index if (duplicates[x]==True)]
bad_features

In [ ]:
X_mi = X.drop(bad_features, axis=1)
X_mi.head()

In [ ]:
# Common dropped features from Chi2 test and MI test
common_bad_features = set(features_to_drop) & set(bad_features)
common_bad_features

### Summary

Of the dropped features between the two modified input data, we had 4 common features. 

One thing I did not consider for the mutual information test was testing for interaction variables. Just because the features were not correlated to the target variable does not mean they can have some relationship with the other features. 

# 3. Model Building

The algorithm we will use for modeling is Random Forest. To increase robustness of the model, I will perform cross validation to get a mean accuracy between 5 folds of training and testing data. Also, all feature selection data sets will be tested, and the performance will be compared.

### Baseline Model

In [ ]:
# Separate training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y[0], test_size=0.25, random_state=0)

# Baseline Random Forest with default parameters
model = RandomForestClassifier(random_state=0)
fit = model.fit(X_train, y_train)
predictions = fit.predict(X_test)
print(classification_report(predictions, y_test))

Interestingly enough, we got an accuracy of 100% which does not seem very reasonable. I will conduct a 5 fold CV on the default model to get a more robust accuracy.

In [ ]:
# 5 fold CV of default Random Forest model
score = cross_val_score(
    model, X, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of base model: ', score.mean())

In [ ]:
# Separate training and testing data for the chi2 features
Xchi_train, Xchi_test, y_train, y_test = train_test_split(X_chi2, y[0], test_size=0.25, random_state=0)

# Chi 2 features model
fit_chi = model.fit(Xchi_train, y_train)
predictions_chi = fit_chi.predict(Xchi_test)
print(classification_report(predictions_chi, y_test))

Again, we have a 100% accuracy but for the chi2 model. Lets try 5-fold CV.

In [ ]:
# 5 fold CV of Chi2 Random Forest model
score = cross_val_score(
    model, X_chi2, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of chi2 model: ', score.mean())

In [ ]:
# 5 fold CV of MI Random Forest model
score = cross_val_score(
    model, X_mi, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of MI model: ', score.mean())

With default parameters for the model, we have a slight improvement in accuracy for the MI input data, while a slight decrease in accuracy for the Chi2 input.

Let's try them with 50 trees now in the random forest.

In [ ]:
model_50 = RandomForestClassifier(n_estimators=50, random_state=0)

# 5 fold CV of default Random Forest model
score = cross_val_score(
    model_50, X, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of base model (50 trees):', score.mean())

# 5 fold CV of Chi2 Random Forest model
score = cross_val_score(
    model_50, X_chi2, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of chi2 model (50 trees):', score.mean())

# 5 fold CV of MI Random Forest model
score = cross_val_score(
    model_50, X_mi, y[0], cv=5, scoring='accuracy'
)
print('Average accuracy of MI model (50 trees):', score.mean())

Even with a random forest with 50 trees, our MI model still comes out on top with the Chi2 on the bottom again.

Lets try to optimize the n_estimators parameter such that we are not under nor over fitting the data.

In [ ]:
n_estimators = [10,20,30,40,50,60,70,80,90,100]
mean_scores = []
for i in n_estimators:
    model = RandomForestClassifier(n_estimators=i, random_state=0)
    score = cross_val_score(
    model, X_mi, y[0], cv=5, scoring='accuracy'
    )
    mean_scores.append(score.mean())
    print('Average accuracy of MI model (',i,'trees):', score.mean())

In [ ]:
mean_scores = pd.Series(mean_scores, index=n_estimators)

# Plot the CV scores
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x = mean_scores.index, y=mean_scores)

ax.set_ylabel('Accuracy')
ax.set_xlabel('# of n_estimators')
ax.set_title('Accuracy of different n_estimators (5 fold CV)')

We got an optimal number of estimators of 50. Now compare the accuracy with different k-folds.

In [ ]:
# MI model CV with different k-folds
accuracies = []
model = RandomForestClassifier(n_estimators=50, random_state=0)
for i in range(5,55,5):
    score = cross_val_score(
    model, X_mi, y[0], cv=i, scoring='accuracy'
    )
    accuracies.append(score.mean())
    
accuracies = pd.Series(accuracies, index=np.arange(5,55,5))

# Plot the CV scores
fig, ax = plt.subplots(figsize=(12,8))

sns.lineplot(x = accuracies.index, y=accuracies)

ax.set_ylabel('Accuracy')
ax.set_xlabel('# of folds')
ax.set_title('Accuracy vs number of k-folds')

# Conclusion

Building a model based on the features selected from Mutual information resulted in the best performance compared to Chi-squared and default. Also, one thing I realize is that I split the training and testing data after modifying the data. Normally I would split the data first, but since I did not really alter any values (imputation), I do not think it would make a difference.


# Question for the Community

I'm under the impression that "leave one out" cross validation is preferred for smaller datasets, however I am getting a 100% accuracy on anything past 25 fold CV. Is this a case of overfitting, or is it just the dataset?